In [1]:
import os
import re
import base64
import io
from PIL import Image
import logging
from typing import List
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, tool, create_react_agent
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
from langchain.hub import pull as hub_pull
from langchain.prompts import load_prompt

In [2]:
load_dotenv()
chat = ChatOpenAI(model="gpt-4o", temperature=0)
react_prompt_template = hub_pull("hwchase17/react")

C:\Users\PedroLustosa\AppData\Local\Temp\ipykernel_25520\3261502151.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model="gpt-4o", temperature=0)
c:\Users\PedroLustosa\AppData\Local\Programs\Python\Python311\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [3]:
@tool
def list_of_files(extension: str) -> list:
    """
    Get a list of all files of a given type in the current working directory.

    Args:
        extension: The extension of the file you want, e.g. jpg or png. Don't use

    Returns:
        A list of filenames with the specified extension.
    """

    # Clean up the extension
    cleaned_extension = extension.replace('"', '').replace('\n', '')

    # Get the current working directory
    current_dir = os.getcwd()

    # List all files in the directory
    files = [f for f in os.listdir(current_dir) if os.path.isfile(os.path.join(current_dir, f)) and f.endswith(f".{cleaned_extension}")]

    return files

In [4]:
@tool
def generate_image_summary(image_path: str) -> str:
    """Generate a summary of the dashboard based on the image."""
    try:
        cleaned_image_path = image_path.replace("'", "").strip()
        image_filename = os.path.basename(cleaned_image_path)
        
        with open(cleaned_image_path, "rb") as image_file:
            image_base64 = base64.b64encode(image_file.read()).decode('utf-8')
        
        prompt = """
        Analyze the dashboard image and provide a comprehensive summary. Address the following points:

        1. Buttons (clickable elements): Identify and describe any buttons present.
        2. Cards (containers): Identify and describe any cards present.
        3. Charts (visual representations): Enumerate and describe each chart, including type and data represented.
        4. Tables/Matrices (tabular data): Identify any tables or matrices and describe their content.
        5. Layout (structure): Describe the overall layout and key elements of the dashboard.
        6. Purpose (function): Infer the main purpose of this dashboard.
        7. Insights (data analysis): Highlight the most powerful insights that can be drawn from this dashboard.

        Provide your analysis in a structured, easy-to-read format using markdown.
        """
        
        llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
        message = HumanMessage(
            content=[
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}"
                    }
                },
            ]
        )
        response = llm.invoke([message])
        
        # Add a header to the summary using just the filename
        summary = f"# Dashboard Analysis for {image_filename}\n\n{response.content}"
        
        return summary
    except FileNotFoundError:
        return f"Error: Image file not found at path: {cleaned_image_path}"
    except Exception as e:
        return f"Error processing image or generating summary: {str(e)}"

In [5]:
@tool
def save_to_markdown(summary: str, filename: str = "dashboard_analysis.md") -> str:
    """
    Save the given content to a markdown file with the specified filename.
    
    Args:
        content: The text content to save in the file.
        filename: The name of the file to save the content in (should end with .md).
                  Defaults to "dashboard_analysis.md" if not provided.
    
    Returns:
        A message indicating success or failure.
    """
    if not filename.endswith('.md'):
        filename += '.md'
    
    try:
        with open(filename, 'w') as file:
            file.write(summary)
        return f"Content successfully saved to {filename}"
    except Exception as e:
        return f"Error saving content to file: {str(e)}"

In [6]:
# Create the tools list
tools = [list_of_files, generate_image_summary, save_to_markdown]

# Now create the agent and executor
agent = create_react_agent(llm=chat, tools=tools, prompt=react_prompt_template)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

def analyze_dashboards() -> str:
    """
    Analyze dashboard images and save summaries.
    
    Returns:
        str: A message indicating the result of the analysis.
    """
    try:
        result = agent_executor.invoke({
            "input": """
            You are a Dashboard Analyzer AI. Your goal is to analyze dashboard images and provide insights.
            Follow these steps:
            1. Check for jpg or png files in the current working directory.
            2. For each image found:
               a. Generate a comprehensive summary.
               b. Include the image filename in the summary.
            3. Combine all summaries into a single markdown file.
            4. Save the combined summaries in a file named 'dashboard_analysis.md'.
            5. If no images are found, return an appropriate message.
            """
        })
        return result['output']
    except Exception as e:
        logging.error(f"An error occurred during dashboard analysis: {str(e)}")
        return f"Error: {str(e)}"



In [7]:
analyze_dashboards()



> Entering new AgentExecutor chain...
To begin the analysis, I need to check for any jpg or png files in the current working directory. 

Action: list_of_files
Action Input: "jpg"[]There are no jpg files in the current working directory. I should now check for png files.

Action: list_of_files
Action Input: "png"['1.Home.png', '2.Visao_Geral.png', '3.Grupo Malls.png', '4.Cenarios.png', '5.Projetos.png']I have found several png files in the current working directory. I will now generate a comprehensive summary for each image and include the image filename in the summary.

Action: generate_image_summary
Action Input: '1.Home.png'# Dashboard Analysis for 1.Home.png

# Dashboard Analysis

## 1. Buttons (Clickable Elements)
- **Visão Geral**: Likely provides an overview or summary of the data related to malls.
- **Grupo Malls**: Possibly focuses on a specific group or category of malls.
- **Cenários**: May present different scenarios or projections related to mall performance.
- **Projeto

"The dashboard images have been analyzed, and the summaries have been saved in 'dashboard_analysis.md'."